In [ ]:
!pip install tensorflow_federated==0.20.0

In [26]:
import nest_asyncio
nest_asyncio.apply()
import sys
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
from tensorflow import keras
from tensorflow.keras import layers 
import tensorflow_federated as tff
tf.compat.v1.enable_eager_execution()
from tensorflow_model_optimization.python.core.internal import tensor_encoding as te


In [20]:
dataset_paths = {
  'client_0': '/content/dev1.csv',
  'client_1': '/content/dev2.csv',
  'client_2': '/content/dev3.csv'
}

for i, (id, path) in enumerate(dataset_paths.items()):
  with open(path, 'w') as f:
    for _ in range(i):
      f.write(f'test,0.0,{i}\n')

record_defaults = ['', 0.0, 0]

@tf.function
def create_tf_dataset_for_client_fn(dataset_path):
   return tf.data.experimental.CsvDataset(
     dataset_path, record_defaults=record_defaults )

source = tff.simulation.datasets.FilePerUserClientData(
  dataset_paths, create_tf_dataset_for_client_fn)

In [21]:
traindata = [source in range(10)]
train_data = tf.reshape(traindata, [1,1])

In [22]:
def model_fn():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(10, tf.nn.softmax, input_shape=(784,),
                            kernel_initializer='zeros')
  ])
  return tff.learning.from_keras_model(
      model,
      input_spec=train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
trainer = tff.learning.algorithms.build_weighted_fed_avg(
  model_fn,
  client_optimizer_fn=lambda: tf.keras.optimizers.SGD(0.1))
state = trainer.initialize()
for _ in range(5):
  result = trainer.next(state, train_data)
  state = result.state
  metrics = result.metrics
  print(metrics['train']['loss'])